In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

In [ ]:
# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyter_loader import JupyterLoader

loader = JupyterLoader()
loader.load_all()

# Human Player 

Next, it should be possible to play against the random engine with human input. 
In order to be compatible with the previously defined `play_game` method, player input is realized as an engine too. 

In the `HumanEngine` class only the `play` method is defined, which takes the current board `board` as a parameter and returns the move the user entered. Additionally, it also allows entering some predefined keywords. If the input is one of the keywords to abort, an exception will be thrown as this will allow exiting from `play_game`. If instead the input is one of the debug keywords, the current board fen is printed and the process repeated. This allows to easily paste the current board in external programs to analyse the current position. If the input is one of the revert keywords, two half moves are reverted. If the input is none of the keywords, it is assumed to be a valid uci move. In this case, the move will be returned, otherwise an error message is printed and the process repeated.

In [ ]:
from converted_notebooks.s04_engine_interface import Engine, ScoredMove
import chess
import IPython.display


class HumanEngine(Engine):
    ABORT_KEYWORDS = ["EXIT", "CANCEL"]
    DEBUG_KEYWORDS = ["FEN", "DEBUG"]
    REVERT_KEYWORDS = ["REVERT", "UNDO"]

    def play(self, board: chess.Board) -> chess.engine.PlayResult:
        whites_turn = board.turn is chess.WHITE
        while True:
            user_input = input(
                f"Enter move for {'white' if whites_turn else 'black'} in UCI notation (special keywords: 'exit', 'fen'): "
            )

            if user_input.upper() in HumanEngine.ABORT_KEYWORDS:
                raise Exception("User aborted")

            if user_input.upper() in HumanEngine.DEBUG_KEYWORDS:
                print(f"FEN: {board.fen()}", flush=True)
                continue

            if user_input.upper() in HumanEngine.REVERT_KEYWORDS:
                # Revert two moves
                board.pop()
                board.pop()
                IPython.display.clear_output(wait=True)
                IPython.display.display(board)
                continue

            try:
                move = chess.Move.from_uci(user_input)
                if move in board.legal_moves:
                    return ScoredMove(0, move)
                print("Illegal input. Please try again.", flush=True)
            except ValueError:
                print(
                    f"Illegal notation. Valid moves are {list(board.legal_moves)}",
                    flush=True
                )

This engine can be used as shown in the following commented block.

In [ ]:
import random
from converted_notebooks.s06_play import play_game
from converted_notebooks.s05_random_engine import RandomEngine

random.seed(42)

board = chess.Board()
play_game(board, HumanEngine(), RandomEngine(), display_board=True)
print(board.outcome())